In [1]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Lasso
data = pd.read_csv('./HW3 附件/HW3_creditcard.csv')

In [10]:
def metrics(y_test, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Calculate precision
    precision = precision_score(y_test, y_pred)
    print("Precision:", precision)

    # Calculate recall (sensitivity)
    recall = recall_score(y_test, y_pred)
    print("Recall (Sensitivity):", recall)

    # Calculate F1 score (harmonic mean of precision and recall)
    f1 = f1_score(y_test, y_pred)
    print("F1 Score:", f1)
    # Calculate ROC AUC (Receiver Operating Characteristic Area Under the Curve)
    roc_auc = roc_auc_score(y_test, y_pred)
    print("ROC AUC:", roc_auc)

In [5]:
df = data.copy()
target_column = 'Class'
test_size = 0.3
random_state = 42

X = df.drop(columns=[target_column])
y = df[target_column]

# Split the data into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=random_state)

In [21]:
print("In all the data:")
print(f"Class 0: {(y == 0).sum()}")
print(f"Class 1: {(y == 1).sum()}")
print('')
print("In Train data:")
print(f"Class 0: {(y_train == 0).sum()}")
print(f"Class 1: {(y_train == 1).sum()}")
print('')
print("In Val data:")
print(f"Class 0: {(y_val == 0).sum()}")
print(f"Class 1: {(y_val == 1).sum()}")

In all the data:
Class 0: 28432
Class 1: 492

In Train data:
Class 0: 19909
Class 1: 337

In Val data:
Class 0: 8523
Class 1: 155


In [8]:
print(X_train.shape)
print(X_val.shape)
print(X.shape)

(20246, 30)
(8678, 30)
(28924, 30)


In [35]:
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)
weighted_tree_classifier = DecisionTreeClassifier(class_weight={0: 1, 1: 2}, random_state=42)


def test_model(classifier):
    classifier.fit(X_train, y_train)
    # Predict on the training data and calculate accuracy
    y_pred = classifier.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    print("Training Accuracy:", accuracy)
    confusion = confusion_matrix(y_train, y_pred)
    print(confusion)
    metrics(y_train, y_pred)


    # Predict on the validation data and calculate accuracy
    y_pred = classifier.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    print("Validation Accuracy:", accuracy)
    confusion = confusion_matrix(y_val, y_pred)
    print(confusion)
    metrics(y_val, y_pred)

test_model(decision_tree_classifier)
test_model(weighted_tree_classifier)


Training Accuracy: 1.0
[[19909     0]
 [    0   337]]
Accuracy: 1.0
Precision: 1.0
Recall (Sensitivity): 1.0
F1 Score: 1.0
ROC AUC: 1.0
Validation Accuracy: 0.9944687716063609
[[8503   20]
 [  28  127]]
Accuracy: 0.9944687716063609
Precision: 0.8639455782312925
Recall (Sensitivity): 0.8193548387096774
F1 Score: 0.8410596026490067
ROC AUC: 0.9085041235669706
Training Accuracy: 1.0
[[19909     0]
 [    0   337]]
Accuracy: 1.0
Precision: 1.0
Recall (Sensitivity): 1.0
F1 Score: 1.0
ROC AUC: 1.0
Validation Accuracy: 0.9942383037566259
[[8502   21]
 [  29  126]]
Accuracy: 0.9942383037566259
Precision: 0.8571428571428571
Recall (Sensitivity): 0.8129032258064516
F1 Score: 0.8344370860927152
ROC AUC: 0.9052196523259644


In [32]:
weighted_tree_classifier.fit(X_train, y_train)

# Predict on the training data and calculate accuracy
y_pred = weighted_tree_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Training Accuracy:", accuracy)
confusion = confusion_matrix(y_train, y_pred)
print(confusion)
metrics(y_train, y_pred)


# Predict on the validation data and calculate accuracy
y_pred = weighted_tree_classifier.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
confusion = confusion_matrix(y_val, y_pred)
print(confusion)
metrics(y_val, y_pred)


Training Accuracy: 1.0
[[19909     0]
 [    0   337]]
Accuracy: 1.0
Precision: 1.0
Recall (Sensitivity): 1.0
F1 Score: 1.0
ROC AUC: 1.0
Validation Accuracy: 0.9942383037566259
[[8502   21]
 [  29  126]]
Accuracy: 0.9942383037566259
Precision: 0.8571428571428571
Recall (Sensitivity): 0.8129032258064516
F1 Score: 0.8344370860927152
ROC AUC: 0.9052196523259644


In [36]:
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# 使用SMOTE生成合成样本
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 创建Decision Tree分类器
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# 使用原始数据拟合模型
decision_tree_classifier.fit(X_train, y_train)

# 使用SMOTE生成的数据拟合模型
decision_tree_classifier_resampled = DecisionTreeClassifier(random_state=42)
decision_tree_classifier_resampled.fit(X_train_resampled, y_train_resampled)

# 预测并计算原始数据模型的指标
y_pred = decision_tree_classifier.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)

# 预测并计算使用SMOTE生成的数据模型的指标
y_pred_resampled = decision_tree_classifier_resampled.predict(X_val)
accuracy_resampled = accuracy_score(y_val, y_pred_resampled)
precision_resampled = precision_score(y_val, y_pred_resampled)
recall_resampled = recall_score(y_val, y_pred_resampled)
f1_resampled = f1_score(y_val, y_pred_resampled)
roc_auc_resampled = roc_auc_score(y_val, y_pred_resampled)

# 打印原始数据模型和使用SMOTE生成数据模型的指标结果
print("Original Data Model:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)

print("\nResampled Data Model:")
print("Accuracy:", accuracy_resampled)
print("Precision:", precision_resampled)
print("Recall:", recall_resampled)
print("F1 Score:", f1_resampled)
print("ROC AUC:", roc_auc_resampled)

# 打印混淆矩阵
print("\nOriginal Data Model Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print("\nResampled Data Model Confusion Matrix:")
print(confusion_matrix(y_val, y_pred_resampled))



Original Data Model:
Accuracy: 0.9944687716063609
Precision: 0.8639455782312925
Recall: 0.8193548387096774
F1 Score: 0.8410596026490067
ROC AUC: 0.9085041235669706

Resampled Data Model:
Accuracy: 0.9891680110624568
Precision: 0.6532663316582915
Recall: 0.8387096774193549
F1 Score: 0.7344632768361582
ROC AUC: 0.9153069682415325

Original Data Model Confusion Matrix:
[[8503   20]
 [  28  127]]

Resampled Data Model Confusion Matrix:
[[8454   69]
 [  25  130]]


In [39]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# 使用SMOTE生成合成样本
smote = SMOTE(sampling_strategy='auto', random_state=42)
xgb_model = xgb.XGBClassifier()

# 创建一个Pipeline，包含SMOTE和XGBoost模型
pipeline = Pipeline([('SMOTE', smote), ('XGBoost', xgb_model)])

# 使用Pipeline拟合模型
pipeline.fit(X_train, y_train)

# 预测并计算模型的指标
y_pred = pipeline.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)

# 打印模型的指标结果
print("XGBoost Model with SMOTE:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)

# 打印混淆矩阵
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


XGBoost Model with SMOTE:
Accuracy: 0.9967734501037105
Precision: 0.9261744966442953
Recall: 0.8903225806451613
F1 Score: 0.9078947368421052
ROC AUC: 0.9445159776392533
Confusion Matrix:
[[8512   11]
 [  17  138]]
